In [4]:
import sys
sys.path.append("../dwaveutils/dwavetools")
sys.path.append("../dwaveutils/probrep")
from dictrep import DictRep
from dwavetools import dense_connect_2000Q
from dwave.system.samplers import DWaveSampler
import networkx as nx
import dwave_networkx as dnx
from collections import defaultdict

### First, get the chip data

In [5]:
sampler = DWaveSampler()
wqubits = sampler.properties['qubits']
wcouplers = sampler.properties['couplers']
chipdata = {'wqubits': wqubits, 'wcouplers': wcouplers}

### Create large logical qubits from dense, rectangular chains and connect them
The way this works is as follows: the dense_connect_2000Q function creates a dense chain connection from some starting unit cell to a final unit cell in a rectangular box with R rows and C columns of unit cells. In particular, it starts the chain with qubit qi, then moves from left to right (up to down) by C (by R) unit cells while chaining nearest neighbor unit cells within the R X C rectangle.

In [9]:
# create the first logical qubit Hamiltonian by chaining 8 x 10 unit cells in upper-left corner
logq1 = dense_connect_2000Q(chipdata, qi = 0, R = 10, C = 8, hval =  'h', Jval = 'J')

Going to try and connect unit cell 0 with unit cell 151
Successfully created a dense chain with 639 qubits.


In [10]:
# create the second logical qubit by chaining 8 x 10 unit cells in top right corner
logq2 = dense_connect_2000Q(chipdata, qi = 64, R = 10, C = 8, hval = 'h', Jval = 'J')

Going to try and connect unit cell 8 with unit cell 159
Successfully created a dense chain with 635 qubits.


In [11]:
# create the third logical qubit by chaining 5 x 16 unit cells at bottom
logq3 = dense_connect_2000Q(chipdata, qi = 1280, R = 5, C = 16, hval = 'h', Jval = 'J')

Going to try and connect unit cell 160 with unit cell 239
Successfully created a dense chain with 637 qubits.


In [12]:
# Now, we connect the logical rectangular qubits with couplings between them
inter_uc_couplings = {(60, 68): 'J2', (1288, 1160): 'J1', (1392, 1264): 'J1'}

In [13]:
# combine them all together
dicts = [logq1, logq2, logq3, inter_uc_couplings]
super_dict = {}
for d in dicts:
    for k, v in d.items():
        super_dict[k].add(v)

KeyError: (0, 0)

In [ ]:
super_dict

In [6]:
# get working qubits and couplers from the solver
sampler = DWaveSampler()
wqubits = sampler.properties['qubits']
wcouplers = sampler.properties['couplers']
print(sampler.properties['chip_id'])

DW_2000Q_3


In [9]:
# create a list of all qubits and find the set difference between working and all
allqubits = [q for q in range(2048)]
brokenqubits = sorted(list(set(allqubits) - set(wqubits)))
print(brokenqubits)

[1, 9, 413, 421, 478, 486, 529, 536, 814, 822, 1099, 1221, 1229, 1429, 1437, 1476, 1484, 1510, 1936, 2032]


In [12]:
# create a list of all couplers and find the set difference between working and all
allcouplers = []
for q in range(2048):
    if q % 8 < 4:
        allcouplers.extend([[q, (q-q%8)+4], [q, (q-q%8)+5], [q, (q-q%8)+6], [q, (q-q%8)+7]])
        if q + 128 < 2047:
            allcouplers.append([q, q+128])
    else:
        if ((q-q%8+4 + 4) % 128) == 0:
            continue
        else:
            allcouplers.append([q, q+8])
brokencouplers = [coupler for coupler in allcouplers if coupler not in wcouplers]
with open ('brokencouplers.txt', 'w') as f:
    for bc in brokencouplers:
        f.write(str(bc[0]) + ', ' + str(bc[1]) + '\n')
print(brokencouplers)

[[1, 4], [1, 5], [1, 6], [1, 7], [1, 129], [9, 12], [9, 13], [9, 14], [9, 15], [9, 137], [90, 93], [401, 529], [405, 413], [408, 413], [408, 536], [409, 413], [410, 413], [411, 413], [413, 421], [416, 421], [417, 421], [418, 421], [418, 422], [419, 421], [421, 429], [470, 478], [472, 478], [473, 478], [474, 478], [475, 478], [478, 486], [480, 486], [481, 486], [482, 486], [483, 486], [486, 494], [529, 532], [529, 533], [529, 534], [529, 535], [529, 657], [536, 540], [536, 541], [536, 542], [536, 543], [536, 664], [806, 814], [808, 814], [809, 814], [810, 814], [811, 814], [814, 822], [816, 822], [817, 822], [818, 822], [819, 822], [822, 830], [971, 1099], [1099, 1100], [1099, 1101], [1099, 1102], [1099, 1103], [1099, 1227], [1173, 1181], [1213, 1221], [1216, 1221], [1217, 1221], [1218, 1221], [1219, 1221], [1221, 1229], [1224, 1229], [1225, 1229], [1226, 1229], [1227, 1229], [1229, 1237], [1421, 1429], [1424, 1429], [1425, 1429], [1426, 1429], [1427, 1429], [1429, 1437], [1432, 1437], 

In [15]:
q1s = [449,563,577,577,577,577,577,691,691,691,691,691,715,977,1004,1008,1009,1010,1011,1012,1105,1105,1105,1105,1105,1235,1235,1268,1272,1273,1274,1275,1602,1648,1730,1730,1730,1730,1730,1776,1776,1776,1776,1776,1858,1858,1858,1858,1858]
q2s = [577,691,705,580,581,582,583,819,692,693,694,695,718,1105,1012,1012,1012,1012,1012,1020,1233,1108,1109,1110,1111,1238,1239,1276,1276,1276,1276,1276,1730,1776,1858,1732,1733,1734,1735,1904,1780,1781,1782,1783,1986,1860,1861,1862,1863]
Bhavikacouplers = [[q1s[i], q2s[i]] for i in range(len(q1s))]

In [20]:
print("Length of (Nic computed broken) couplers is: {}".format(len(brokencouplers)))
print("Length of Bhavika computed compuers is: {}".format(len(Bhavikacouplers)))

Length of (Nic computed broken) couplers is: 49
Length of Bhavika computed compuers is: 49


In [28]:
diff1 = [coupler for coupler in Bhavikacouplers if coupler not in brokencouplers]
print(diff1)
diff2 = [coupler for coupler in brokencouplers if coupler not in Bhavikacouplers]
print(diff2)

[]
[]
